# Prepare a Subset of Amazon Reviews for Class

Data is a partially processed version of the Home and Kitchen category in the Amazon Reviews dataset (see info below).

In [93]:
from IPython.display import display, Markdown
with open("../Data-AmazonReviews/Amazon Product Reviews.md") as f:
    display(Markdown(f.read()))

# Amazon Product Reviews

- URL: https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews 

## Description

This is a large crawl of product reviews from Amazon. This dataset contains 82.83 million unique reviews, from around 20 million users.

## Basic statistics

| Ratings:  | 82.83 million        |
| --------- | -------------------- |
| Users:    | 20.98 million        |
| Items:    | 9.35 million         |
| Timespan: | May 1996 - July 2014 |

## Metadata

- reviews and ratings
- item-to-item relationships (e.g. "people who bought X also bought Y")
- timestamps
- helpfulness votes
- product image (and CNN features)
- price
- category
- salesRank

## Example

```
{  "reviewerID": "A2SUAM1J3GNN3B",  "asin": "0000013714",  "reviewerName": "J. McDonald",  "helpful": [2, 3],  "reviewText": "I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!",  "overall": 5.0,  "summary": "Heavenly Highway Hymns",  "unixReviewTime": 1252800000,  "reviewTime": "09 13, 2009" }
```

## Download link

See the [Amazon Dataset Page](https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/) for download information.

The 2014 version of this dataset is [also available](https://cseweb.ucsd.edu/~jmcauley/datasets/amazon/links.html).

## Citation

Please cite the following if you use the data:

**Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering**

R. He, J. McAuley

*WWW*, 2016
[pdf](https://cseweb.ucsd.edu/~jmcauley/pdfs/www16a.pdf)

**Image-based recommendations on styles and substitutes**

J. McAuley, C. Targett, J. Shi, A. van den Hengel

*SIGIR*, 2015
[pdf](https://cseweb.ucsd.edu/~jmcauley/pdfs/sigir15.pdf)

# Load the Data

We will download our **corpus** of reviews.

In [94]:
## Import necessary packages
import pandas as pd
import nltk
import os, glob, json

In [95]:
sorted(os.listdir("../Data-AmazonReviews/"))

['Amazon Product Reviews.md',
 'amazon-reviews-home-ktichen_hoover.csv',
 'amazon-reviews-video-games-combined.csv.gz',
 'amazon-reviews-with-metadata-home-kitchen.csv.gz',
 'processed-reviews-hoover.csv',
 'processed-reviews-hoover.joblib']

In [96]:
# Select which raw reviews file to use
fpath_reviews = "../Data-AmazonReviews/amazon-reviews-with-metadata-home-kitchen.csv.gz"
# fpath_reviews = "../Data-AmazonReviews/amazon-reviews-video-games-combined.csv.gz

In [97]:
## Load full corpus
df_full = pd.read_csv(fpath_reviews)
df_full.head(10)

,asin,reviewerID,date,reviewText,summary,overall,title,brand,main_cat,description,feature,category,imageURL
0,B000067DW6,A6VPIOPMDJ8H7,2005-05-28,"In my recently deleted review, I praised these PUR filters for their performance and the Automatic Safety Monitor. Removing the Monitor has now made the filter much less reliable.\n\nPUR claims in an email reply that consumers prefer the calendar sticker to the Automatic Safety Monitor:\n\n""Hi ...",A shadow of a once-great product,2.0,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water",PUR,Amazon Home,"['Replacement Filter Cartridge, 2-11/16x2-11/16x7H, Blue (PUR70322)', 'PUR CRF950Z Ultimate Replacement Filter For PitcherPUR CRF950Z Ultimate Replacement Filter For Pitcher Features: Fits all Pur water pitchers CR1500R-15002 (1653-3085) Reduces harmful chemicals linked to cancer Reduces pestici...","[""PUR BASIC WATER FILTER REPLACEMENT: PUR's Basic pitcher filters are certified to reduce 22 contaminants, so you know you are drinking cleaner, great-tasting water. All PUR pitcher filters fit in all PUR pitchers and dispensers"", 'PITCHER WATER FILTERS: PUR Basic Pitcher filters provide 40 gall...","['Home & Kitchen', 'Kitchen & Dining', 'Water Coolers & Filters', 'Replacement Water Filters', 'Replacement Under-Sink Water Filters']",['https://images-na.ssl-images-amazon.com/images/I/41kvz5hCbaL._SS40_.jpg']
1,B000067DW6,A2CBZMETQJTNEE,2005-01-01,"I lived in Peru, in the Andes where it was imperative to have filtered water or buy expensive bottle water. I had the town water and it had fecal contamination but back then the Pur 2 Stage filter would filter out these pathogens, not now. Now, P&G has cheapen the filter and at best it should ...","What once was a good filter is now, basicly, worthless.",1.0,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water",PUR,Amazon Home,"['Replacement Filter Cartridge, 2-11/16x2-11/16x7H, Blue (PUR70322)', 'PUR CRF950Z Ultimate Replacement Filter For PitcherPUR CRF950Z Ultimate Replacement Filter For Pitcher Features: Fits all Pur water pitchers CR1500R-15002 (1653-3085) Reduces harmful chemicals linked to cancer Reduces pestici...","[""PUR BASIC WATER FILTER REPLACEMENT: PUR's Basic pitcher filters are certified to reduce 22 contaminants, so you know you are drinking cleaner, great-tasting water. All PUR pitcher filters fit in all PUR pitchers and dispensers"", 'PITCHER WATER FILTERS: PUR Basic Pitcher filters provide 40 gall...","['Home & Kitchen', 'Kitchen & Dining', 'Water Coolers & Filters', 'Replacement Water Filters', 'Replacement Under-Sink Water Filters']",['https://images-na.ssl-images-amazon.com/images/I/41kvz5hCbaL._SS40_.jpg']
2,B000067DW6,A2ZR3YTMEEIIZ4,2003-10-03,"Pur makes water filters for campers and backpackers, proving their abilities with water far worse and more suspect than your home tap. Only Katadyn has a better reputation for personal filtration, and Katadyn bought Pur. Rest assured, these are technologically solid and proven filters.\nSo why...","Pur over Brita, and neither softens water.",5.0,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water",PUR,Amazon Home,"['Replacement Filter Cartridge, 2-11/16x2-11/16x7H, Blue (PUR70322)', 'PUR CRF950Z Ultimate Replacement Filter For PitcherPUR CRF950Z Ultimate Replacement Filter For Pitcher Features: Fits all Pur water pitchers CR1500R-15002 (1653-3085) Reduces harmful chemicals linked to cancer Reduces pestici...","[""PUR BASIC WATER FILTER REPLACEMENT: PUR's Basic pitcher filters are certified to reduce 22 contaminants, so you know you are drinki

In [98]:
# Keep only the required columns
use_cols = ['reviewText','summary','overall','brand','title']
df_full = df_full[use_cols]
df_full

,reviewText,summary,overall,brand,title
0,"In my recently deleted review, I praised these PUR filters for their performance and the Automatic Safety Monitor. Removing the Monitor has now made the filter much less reliable.\n\nPUR claims in an email reply that consumers prefer the calendar sticker to the Automatic Safety Monitor:\n\n""Hi ...",A shadow of a once-great product,2.0,PUR,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water"
1,"I lived in Peru, in the Andes where it was imperative to have filtered water or buy expensive bottle water. I had the town water and it had fecal contamination but back then the Pur 2 Stage filter would filter out these pathogens, not now. Now, P&G has cheapen the filter and at best it should ...","What once was a good filter is now, basicly, worthless.",1.0,PUR,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water"
2,"Pur makes water filters for campers and backpackers, proving their abilities with water far worse and more suspect than your home tap. Only Katadyn has a better reputation for personal filtration, and Katadyn bought Pur. Rest assured, these are technologically solid and proven filters.\nSo why...","Pur over Brita, and neither softens water.",5.0,PUR,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water"
3,"We've tried both flavors. Pur's basic filter works better than their lead version. The lead filter lasts just about one month, even after following their break-in instructions. And it was excruciatingly slow, even when brand new. Buy their basic filter.","Pur's Lead Filter Takes Forever To Work, Doesn't Last Long",2.0,PUR,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water"
4,Finally got around to using these and they are horrible. So far 2 out of the 3 we used leaked filtration material into our water.,LEAK,1.0,PUR,"PUR Basic Water Pitcher Replacement Filter, 2-Stage, 1-Pack, Filter Replacements for PUR Water Filter Pitchers, Reduced Chlorine Taste and Odor, Filters Provide 40 Gallons/2 Months of Filtered Water"
...,...,...,...,...,...
110207,This tumbler is just as good as a Yeti tumbler but better because it's half the price. I got 5 of them and plan on buying more. I'm planning on getting some as Christmas gifts this year since everyone I know wants mine. Everything they say about them is true! Great product and it should last...,Amazing product and one that anyone would appreciate if you choose to get as a gift.,5.0,RTIC,RTIC 30 oz Stainless Steel Tumbler Cup w/ Splash Proof Lid
110208,This is an amazing cup. Use it daily. Drapela Designs did etching for me and i get so many questions about this cup. Keeps drink cold until there is no more to drink. would give 10 stars if it were possible.,Cold forever,5.0,RTIC,RTIC 30 oz Stainless Steel Tumbler Cup w/ Splash Proof Lid
110209,I bought two of these and they work great. Just as good as the more expensive competition. Don't waste your money when you can get these for half the price.,Keeps Ice All Day...and Longer,5.0,RTIC,RTIC 30 oz Stainless Steel Tumbler Cup w/ Splash Proof Lid
110210,"Buy direct from RTIC, awesome cup!",Love!,5.0,RTIC,RTIC 30 oz Stainless Steel Tumbler Cup w/ Splash Proof Lid


# Some light EDA

In [99]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110212 entries, 0 to 110211
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   reviewText  110212 non-null  object 
 1   summary     110189 non-null  object 
 2   overall     110212 non-null  float64
 3   brand       110212 non-null  object 
 4   title       110212 non-null  object 
dtypes: float64(1), object(4)
memory usage: 4.2+ MB


In [100]:
# Check duplicatges
df_full.duplicated().sum()

5486

In [101]:
# # Drop duplicates
# df_full = df_full.drop_duplicates()
# df_full.duplicated().sum()

In [102]:
# # Combine the 
# df_full['text'] = df_full['summary'] + ": " + df_full['reviewText']
# df_full

In [103]:
# df_full['title-short'] = df_full['title'].map(lambda x: x[:40])

### Selecting a Subset

In [104]:
df_full['brand'].value_counts()

Instant Pot              9027
Thermos                  8957
Hoover                   8257
Contigo                  7443
Cuisinart                5960
Pinzon by Amazon         5947
New Metro Design         5825
Ozeri                    4547
Hamilton Beach           3991
Clara Clark              3962
Coop Home Goods          3721
Bissell                  3434
RTIC                     3345
InterDesign              3285
Aroma Housewares         3248
HC COLLECTION            3144
Zinus                    2768
Himalayan Glow           2720
Eureka                   2620
Lodge                    2543
Zyliss                   2288
BLACK+DECKER             2228
Presto                   2222
PUR                      2045
Keurig                   1940
SleepBetter              1761
American Weigh Scales    1528
Swissmar                 1456
Name: brand, dtype: int64

In [105]:
overall_by_brand_counts = df_full.groupby('brand')['overall'].value_counts().unstack(1)
overall_by_brand_counts.head()

overall,1.0,2.0,3.0,4.0,5.0
brand,,,,,
American Weigh Scales,78,49,87,209,1105
Aroma Housewares,194,165,202,525,2162
BLACK+DECKER,126,94,143,337,1528
Bissell,175,154,188,493,2424
Clara Clark,209,192,350,521,2690


In [106]:
# Finding brands with the most one-star reviews
overall_by_brand_counts.sort_values(1.0, ascending=False)

overall,1.0,2.0,3.0,4.0,5.0
brand,,,,,
New Metro Design,613,354,415,715,3728
Cuisinart,561,278,378,786,3957
Hoover,531,281,437,1397,5611
Thermos,399,306,506,1054,6692
Contigo,389,340,496,941,5277
Hamilton Beach,358,256,365,759,2253
InterDesign,299,259,442,529,1756
Instant Pot,271,147,275,738,7596
Pinzon by Amazon,262,276,539,991,3879


In [107]:
SELECTED_BRAND  ="Hoover"
new_column_order = [
    #'text'.
    'reviewText',
    'summary',    
    'overall',
    'brand',
    'title']

df = df_full.loc[ df_full['brand']==SELECTED_BRAND, new_column_order]
# Shuffling Rows & Reset Index
df = df.sample(frac=1, replace=False)
df = df.reset_index(drop=True)
df = df.copy()
df

,reviewText,summary,overall,brand,title
0,"Carpets are clean and dry within one day. Depending on the weather, carpets may dry within an hour or two. Suction is amazing. I ran out of the Hoover cleaning solution mid-clean and substituted ""Kids n Pets"" cleaner with good results. No foaming, no shutdown, carpet just as clean as portion cle...",I love it.,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
1,"I have been buying these units for years. My last one cleaned well, most due but are really heavy and cumbersome and if you don't have the clean and dirty water tanks on the machine fit just perfectly nothing works. It was a real hassle making sure these tanks were on and all kinds of latches et...",Wished I need two!,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
2,"I cleaned my carpets for the first time in 4 years today, yuck. Even without shoes dirt still gets them. I did 2 passes on all my rugs and there was a huge difference. I knew they would never look new again but they look impressively better. The welcome mat is a match to my runner rugs and i...",Cleaned up great,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
3,"Purchased this shampooer the replace the one that could nver finds it way home, this one stays home!!\nThe many improvements over my other one (bought in 2005) are great, much lighter, the foot release works better. The water tanks-no big improvement. having the option of a seperate cleaner t...",Another great Hoover product,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
4,"This machine is the best I've ever used...better than those you rent and not nearly as heavy. I like that it is set up to rinse after the soap. It suctions out so well, I could put my furniture back the same day. Great product for a great price...and thanks to my Prime membership, I paid no shi...",Best Machine Ever!,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
...,...,...,...,...,...
8252,I love this vacuum. I have all hardwood floors and some rugs. It works wonderfully. My days of sweeping and dust mopping are slim to none. It is easy to use. Holds charge for quite a while. It is light weight so it's easy to transport up and down stairs. I have two minor complains. The dust bin ...,I love this vacuum,4.0,Hoover,"Hoover Linx Cordless Stick Vacuum Cleaner, BH50010"
8253,just doesn't do the job.,Two Stars,2.0,Hoover,"Hoover Linx Cordless Stick Vacuum Cleaner, BH50010"
8254,"Overall I am pretty happy with this cleaner. It isn't super heavy when full, and although you have to fill up/empty the resevoirs fairly frequently it isn't too annoying. If the resevoirs were bigger (which some people said they would like) it would just mean the unit would be heavier and bigg...",Good Carpet Cleaner,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
8255,This does not replace professional carpet cleaning at my vacation rental but works well for spot treatment. The reservoir does not hold much and there is s lot of filling and emptying even for an average size room. It's easy to use and clean and I would buy it again as a supplement to the pro c...,Good For Small Areas,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900


In [108]:
df['overall'].value_counts()

5.0    5611
4.0    1397
1.0     531
3.0     437
2.0     281
Name: overall, dtype: int64

In [109]:
# pd.set_option('display.max_colwidth',300)

In [110]:
df.head()

,reviewText,summary,overall,brand,title
0,"Carpets are clean and dry within one day. Depending on the weather, carpets may dry within an hour or two. Suction is amazing. I ran out of the Hoover cleaning solution mid-clean and substituted ""Kids n Pets"" cleaner with good results. No foaming, no shutdown, carpet just as clean as portion cle...",I love it.,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
1,"I have been buying these units for years. My last one cleaned well, most due but are really heavy and cumbersome and if you don't have the clean and dirty water tanks on the machine fit just perfectly nothing works. It was a real hassle making sure these tanks were on and all kinds of latches et...",Wished I need two!,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
2,"I cleaned my carpets for the first time in 4 years today, yuck. Even without shoes dirt still gets them. I did 2 passes on all my rugs and there was a huge difference. I knew they would never look new again but they look impressively better. The welcome mat is a match to my runner rugs and i...",Cleaned up great,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
3,"Purchased this shampooer the replace the one that could nver finds it way home, this one stays home!!\nThe many improvements over my other one (bought in 2005) are great, much lighter, the foot release works better. The water tanks-no big improvement. having the option of a seperate cleaner t...",Another great Hoover product,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
4,"This machine is the best I've ever used...better than those you rent and not nearly as heavy. I like that it is set up to rinse after the soap. It suctions out so well, I could put my furniture back the same day. Great product for a great price...and thanks to my Prime membership, I paid no shi...",Best Machine Ever!,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150


In [111]:
## Save the processed data
fpath_final ='../Data-AmazonReviews/amazon-reviews-home-ktichen_hoover.csv'
df.to_csv(fpath_final, index=False)

In [112]:
pd.read_csv(fpath_final)

,reviewText,summary,overall,brand,title
0,"Carpets are clean and dry within one day. Depending on the weather, carpets may dry within an hour or two. Suction is amazing. I ran out of the Hoover cleaning solution mid-clean and substituted ""Kids n Pets"" cleaner with good results. No foaming, no shutdown, carpet just as clean as portion cle...",I love it.,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
1,"I have been buying these units for years. My last one cleaned well, most due but are really heavy and cumbersome and if you don't have the clean and dirty water tanks on the machine fit just perfectly nothing works. It was a real hassle making sure these tanks were on and all kinds of latches et...",Wished I need two!,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
2,"I cleaned my carpets for the first time in 4 years today, yuck. Even without shoes dirt still gets them. I did 2 passes on all my rugs and there was a huge difference. I knew they would never look new again but they look impressively better. The welcome mat is a match to my runner rugs and i...",Cleaned up great,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
3,"Purchased this shampooer the replace the one that could nver finds it way home, this one stays home!!\nThe many improvements over my other one (bought in 2005) are great, much lighter, the foot release works better. The water tanks-no big improvement. having the option of a seperate cleaner t...",Another great Hoover product,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
4,"This machine is the best I've ever used...better than those you rent and not nearly as heavy. I like that it is set up to rinse after the soap. It suctions out so well, I could put my furniture back the same day. Great product for a great price...and thanks to my Prime membership, I paid no shi...",Best Machine Ever!,5.0,Hoover,Hoover Power Scrub Deluxe Carpet Washer FH50150
...,...,...,...,...,...
8252,I love this vacuum. I have all hardwood floors and some rugs. It works wonderfully. My days of sweeping and dust mopping are slim to none. It is easy to use. Holds charge for quite a while. It is light weight so it's easy to transport up and down stairs. I have two minor complains. The dust bin ...,I love this vacuum,4.0,Hoover,"Hoover Linx Cordless Stick Vacuum Cleaner, BH50010"
8253,just doesn't do the job.,Two Stars,2.0,Hoover,"Hoover Linx Cordless Stick Vacuum Cleaner, BH50010"
8254,"Overall I am pretty happy with this cleaner. It isn't super heavy when full, and although you have to fill up/empty the resevoirs fairly frequently it isn't too annoying. If the resevoirs were bigger (which some people said they would like) it would just mean the unit would be heavier and bigg...",Good Carpet Cleaner,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
8255,This does not replace professional carpet cleaning at my vacation rental but works well for spot treatment. The reservoir does not hold much and there is s lot of filling and emptying even for an average size room. It's easy to use and clean and I would buy it again as a supplement to the pro c...,Good For Small Areas,5.0,Hoover,Hoover Carpet Cleaner SteamVac with Clean Surge Carpet Cleaner Machine F5914900
